# Инициализация пакетов

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
#df = pd.read_excel('data/sample.xlsx')
df = pd.read_csv('data/color_idx.csv', sep='$')

/tmp/ipykernel_17232/3821862338.py:2: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/color_idx.csv', sep='$')


# Добавление новы переменных

In [26]:
# добавление переменной повторное обращение
df['repeated'] = df['Вид выполненных работ'].map(lambda i: 'повторное обращение' in i.lower())
# добавление переменной невыполненных работ
df['unfinished'] = df['Отзыв'].astype('str').map(lambda i: 'работы не выполнены' in i.lower())

In [6]:
# оценка жителей
# признак инцидента
# травм. опасные случаи
# Отзыв
# Оценка качества выполнения работ
# Код оценки качества выполнения ра

# Разница по времени в минутах между началом и закрытием

In [7]:
def count_time_dif(strart_date, end_date):
    return (end_date - strart_date).total_seconds()/60

created_date = df['Дата создания заявки в формате Timezone']
beginin_date = df['Дата начала действия версии заявки в формате Timezone']
closure_date = df['Дата закрытия']

# removing endings
# created_date = created_date.map(lambda t_str: t_str[:t_str.index(".")])
# beginin_date = beginin_date.map(lambda t_str: t_str[:t_str.index(".")])
# closure_date = closure_date.map(lambda t_str: t_str[:t_str.index(".")])

# encoding to datetime
created_date = created_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")) 
beginin_date = beginin_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")) 
closure_date = closure_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")) 

df['Дата создания заявки в формате Timezone'] = created_date
df['Дата начала действия версии заявки в формате Timezone'] = beginin_date
df['Дата закрытия'] = closure_date

In [ ]:
# подсчёт разницы во времени
df['created_time_diff'] = df.apply(lambda x: count_time_dif(x['Дата создания заявки в формате Timezone'], x['Дата закрытия']), axis=1)
df['beginin_time_diff'] = df.apply(lambda x: count_time_dif(x['Дата начала действия версии заявки в формате Timezone'], x['Дата закрытия']), axis=1)

# Критерии анамальности

In [85]:
def find_close_request(df, bnd_time_dif = 72*60):
    created = False
    res = pd.DataFrame(columns = df.columns)
    for i in range(df.shape[0]):
        for j in range(i):
            open_idx = df.columns.get_loc('Дата создания заявки в формате Timezone')
            close_idx = df.columns.get_loc('Дата закрытия')
            time_dif = count_time_dif(df.iloc[j, open_idx], df.iloc[i, close_idx])
            if time_dif < bnd_time_dif:
                if not created:
                    res = df.append(df.iloc[j, :], ignore_index=True)
                    created = True
                else:
                    res = df.append(df.iloc[j, :], ignore_index=True)
    if created:
        return res

def get_incidents(df, get_filter_mask):
  res = df[get_filter_mask(df)]
  res = df.groupby(["Адрес проблемы", "Наименование дефекта"]).filter(lambda x: len(x)>1)
  res = res.groupby(["Адрес проблемы", "Наименование дефекта"], group_keys=False).apply(lambda x: find_close_request(x))
  return res

## 1 Заявки закрытые без выполнения

статус: закрытые/закрытые через МАРМ

тип дефектов все кроме: 
    
    - Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала),
    - Подача документов о поверке ИПУ воды в электронном виде

результативность кроме: Выполнено

результат закрытия кроме: Аварийное/плановое отключение

тип дефекта один и тот же

In [10]:
def get_filter_mask_1(df):
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Наименование дефекта'].astype('str')
  mask = np.logical_and( mask, 
                        active_col.map(lambda i:  not ( ('ввод в эксплуатацию ипу воды' in i.lower()) or 
                                                        ('подача документов о поверке' in i.lower()) ) )
                      )

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' not in i)
                      )

  active_col = df['Вид выполненных работ']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Аварийное/плановое отключение' not in i)
                      )
  return mask

In [12]:
anom_1 = get_incidents(df, get_filter_mask_1)
anom_1.shape

/tmp/ipykernel_16620/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 73)

## 2 Заявки в статусе "закрытые" но на деле нет

статус: закрытые/закрытые через МАРМ

тип дефектов все кроме: 
    
    - Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала),
    - Подача документов о поверке ИПУ воды в электронном виде

результативность: Выполнено

отсутствует: Кол-во возвратов на доработку

срок выполнения: >10

один и тот же адресс и разница между временем заявки (Наименование дефекта)

In [11]:
def get_filter_mask_2(df):
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Наименование дефекта'].astype('str')
  mask = np.logical_and( mask, 
                        active_col.map(lambda i:  not ( ('ввод в эксплуатацию ипу воды' in i.lower()) or 
                                                        ('подача документов о поверке' in i.lower()) ) )
                      )

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' in i)
                      )

  active_col = df['created_time_diff']
  mask = np.logical_and(mask,
          active_col.map(lambda i: i>10))

  return mask

def get_incidents_2(df):
  res = df[get_filter_mask_2(df)]
  res = df.groupby(["Адрес проблемы", "Наименование дефекта"]).filter(lambda x: len(x)>1)
  res = res.groupby(["Адрес проблемы", "Наименование дефекта"], group_keys=False).apply(lambda x: find_close_request(x))
  return res


In [13]:
anom_2 = get_incidents(df, get_filter_mask_2)
anom_2.shape

/tmp/ipykernel_16620/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 73)

## 3 Быстрые невыполненные заявки
статус: закрытые/закрытые через МАРМ

результативность: Выполнено

отсутствует: Кол-во возвратов на доработку

срок выполнения: <10

при этом работа требует меньше 10 (из приложения 2)

один и тот же адресс и разница между временем заявки (Наименование дефекта)

In [86]:
def get_filter_mask_3(df):
  app_2 = pd.read_csv('data/extensions/app_2.csv', sep = '$')        
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' in i))

  active_col = df['Кол-во возвратов на доработку']
  mask = np.logical_and(mask,
          active_col.map(lambda i: np.isnan(i)))

  active_col = df['created_time_diff']
  mask = np.logical_and(mask,
          active_col.map(lambda i: i<10))
  
  active_col = df['Наименование дефекта']
  mask = np.logical_and(mask, 
                active_col.isin(app_2['тип дефекта']))

  return mask

In [87]:
anom_3 = get_incidents(df, get_filter_mask_3)
anom_3.shape

/tmp/ipykernel_17232/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 76)

## 4 Слишком быстрые заявки

результативность: Выполнено

срок выполнения: <10
при этом работа требует больше 10

количество возвратов на доработку: пусто


In [88]:
def get_filter_mask_4(df):
  app_2 = pd.read_csv('data/extensions/app_2.csv', sep = '$')                
  active_col = df['Результативность']
  mask = active_col.map(lambda i: 'Выполнено' in i)

  active_col = df['Кол-во возвратов на доработку']
  mask = np.logical_and(mask,
          active_col.map(lambda i: np.isnan(i)))

  active_col = df['created_time_diff']
  mask = np.logical_and(mask,
          active_col.map(lambda i: i<10))
  
# вставить приложение про больше 10 мин
  active_col = df['Наименование дефекта']
  mask = np.logical_and(mask, 
                ~active_col.isin(app_2['тип дефекта']))

  return mask

In [89]:
anom_4 = get_incidents(df, get_filter_mask_4)
anom_4.shape

/tmp/ipykernel_17232/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 76)

## 5 возвращённые заявки

статус: закрытые/закрытые через МАРМ

Кол-во возвратов на доработку: не пусто и не 0

Отобрать наименование дефектов из приложения 4

Результативность: выполнено

In [94]:
def get_filter_mask_5(df):
  app_4 = pd.read_csv('data/extensions/app_4.csv', sep='$')
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' in i))

  active_col = df['Кол-во возвратов на доработку']
  mask = np.logical_and(mask,
          active_col.map(lambda i: not (np.isnan(i) or i==0) ))

  active_col = df['Идентификатор дефекта']
  mask = np.logical_and(mask, 
                active_col.isin(app_4['Идентификатор']))

  return mask

In [95]:
anom_5 = get_incidents(df, get_filter_mask_5)
anom_5.shape

/tmp/ipykernel_17232/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 76)

## соединение пунктов

In [99]:
bad_mes = pd.concat([anom_1, anom_2, anom_3, anom_4, anom_5])
bad_mes.drop_duplicates(inplace=True)

In [115]:
df.loc[df['Корневой ИД заявки'].isin(bad_mes['Корневой ИД заявки']), 'color'] = 2

In [117]:
df.to_csv('data/color_idx.csv', index=False, sep = '$')

# Считывание координат

In [17]:
geo_data = pd.read_csv('data_sanya.csv', sep='|', skiprows=1, low_memory=False)

In [19]:
geo_data = geo_data[['Упрощённое написание адреса или описание местоположения', 'geodata_center']]
geo_data.dropna(axis='index', inplace=True)
geo_data.head()

,Упрощённое написание адреса или описание местоположения,geodata_center
0,"Косинская улица, дом 26А","{""coordinates"":[37.8281893944955,55.7174827851..."
8,"улица Панфилова, дом 11, строение 1","{""coordinates"":[37.4918057069316,55.8077713279..."
26,"9-я Парковая улица, дом 43/26","{""coordinates"":[37.7984137523718,55.8022031297..."
34,"Фрязевская улица, дом 9, корпус 1","{""coordinates"":[37.822531843712,55.74627944075..."
45,"улица Верхние Поля, дом 26А","{""coordinates"":[37.7539178129885,55.6661694512..."


In [53]:
def get_coordinates(adress):
    coord = geo_data[geo_data.iloc[:,0] == adress]['geodata_center']
    try:
        coord = coord.iloc[0]
        return coord[coord.index("[")+1: coord.index("]")]
    except:
        return np.nan

df['coordinates'] = df['Адрес проблемы'].map(get_coordinates)


In [54]:
df.to_csv('data/with_coordinates.csv', index=False, sep = '$')

# Добавление новых таблиц

In [55]:
df[['x', 'y']] = df['coordinates'].str.split(',', 1, expand=True)

/tmp/ipykernel_17232/1925870112.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['x', 'y']] = df['coordinates'].str.split(',', 1, expand=True)


## ЖКХ

In [29]:
df['color'] = df['color'].astype('int')

In [64]:
df['x'] = df['x'].astype('float')
df['y'] = df['y'].astype('float')

In [71]:
color_grouped = df.groupby(['Наименование управляющей компании'])['color'].agg('sum')
coord_grouped = df.groupby(['Наименование управляющей компании'])['x', 'y'].agg('mean')
res = pd.concat([coord_grouped, color_grouped], axis=1)

/tmp/ipykernel_17232/3715249134.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  coord_grouped = df.groupby(['Наименование управляющей компании'])['x', 'y'].agg('mean')


In [72]:
res.to_csv('data/GBU.csv', sep='$')

## Район

In [74]:
color_grouped = df.groupby(['Наименование района'])['color'].agg('sum')
coord_grouped = df.groupby(['Наименование района'])['x', 'y'].agg('mean')
res = pd.concat([coord_grouped, color_grouped], axis=1)

/tmp/ipykernel_17232/3553592071.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  coord_grouped = df.groupby(['Наименование района'])['x', 'y'].agg('mean')


In [73]:
# res = df.groupby('Наименование района')['color'].value_counts().unstack().fillna(0)
# res.head()

In [77]:
res.to_csv('data/district.csv', sep='$')